# Limpieza y exploración

# CREDENCIALES DE PROYECTO, IMPORTACION DE LIBRERIAS INICIALES

In [1]:
#Importación de station.csv file
#Importación de functions pyspark SQL
from pyspark.sql.functions import*
import ibmos2spark, os

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_5fc9833f13ed4170a3d793ba6b80e11a = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_5fc9833f13ed4170a3d793ba6b80e11a = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

credentials = {
    'endpoint': endpoint_5fc9833f13ed4170a3d793ba6b80e11a,
    'service_id': 'iam-ServiceId-a9f1577d-5270-4858-8b14-345ee4bfb744',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'SZLYLSD_wZpkWnqiF6AzCgVrL9lm-JCubzDrqmhf30Za'
}

configuration_name = 'os_5fc9833f13ed4170a3d793ba6b80e11a_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
#Definir nombre de dataframe con convencion df_ al inicio
df_stations_raw = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('InferSchema','true')\
  .option('header', 'true')\
  .load(cos.url('station.csv', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))
#Se muestra una sola fila de ejemplo para ver que cargo la data
df_stations_raw.take(1)

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210411232431-0000
KERNEL_ID = cb919409-ba87-4fbf-8605-bf6e9245f2ad


[Row(station_id='BT-01', name='3rd Ave & Broad St', lat=47.618418, long=-122.350964, install_date='10/13/2014', install_dockcount=18, modification_date=None, current_dockcount=18, decommission_date=None)]

# INICIO TRABAJO STATIONS

In [2]:
#Mostrar tipo de datos del df_statiosn
#Se identifica que las columnas de tipo date en su lugar son de tipo string: modification_date, decommission_date e install_date
df_stations_raw.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- install_date: string (nullable = true)
 |-- install_dockcount: integer (nullable = true)
 |-- modification_date: string (nullable = true)
 |-- current_dockcount: integer (nullable = true)
 |-- decommission_date: string (nullable = true)



In [3]:
#Se corrobora con el describe el resumen de los datos del df_stations, encontrando que todas las columanas tiene 58 valores "válidos" a excepción de dos columnas.
#En específico modification_date y decommission date tienen valores nulos, esto se entiende que ambos sucesos pueden no necesariamente haber ocurrido en las estaciones.
df_stations_raw.describe().show()

+-------+----------+--------------------+-----------------+--------------------+------------+------------------+-----------------+------------------+-----------------+
|summary|station_id|                name|              lat|                long|install_date| install_dockcount|modification_date| current_dockcount|decommission_date|
+-------+----------+--------------------+-----------------+--------------------+------------+------------------+-----------------+------------------+-----------------+
|  count|        58|                  58|               58|                  58|          58|                58|               17|                58|                4|
|   mean|      null|                null|47.62479551586206| -122.32724190862069|        null|17.586206896551722|             null|16.517241379310345|             null|
| stddev|      null|                null| 0.01906576274897|0.014956664123887123|        null| 3.060985294637446|             null| 5.117021403705188|           

In [4]:
#Se visualizan los datos de las 3 columnas modification_date, decommission_date e install_date debido a su tipo de dato que actualmente es string
#En este caso se realiza un show de exactamente el count de registros que hay en el dataframe. En caso de ser un dataframe mas grande esto no seria para nada optimo.
#De esta manera podemos ver todos los registros pudiendo comprobar el formato que en este caso encontramos dos MM/DD/YYYY y M/DD/YYYY.
df_stations_raw.select("install_date","modification_date", "decommission_date").show(df_stations_raw.count())

+------------+-----------------+-----------------+
|install_date|modification_date|decommission_date|
+------------+-----------------+-----------------+
|  10/13/2014|             null|             null|
|  10/13/2014|             null|             null|
|  10/13/2014|             null|             null|
|  10/13/2014|             null|             null|
|  10/13/2014|             null|             null|
|   7/27/2015|             null|             null|
|  10/13/2014|             null|             null|
|  10/13/2014|        11/9/2015|             null|
|  10/13/2014|             null|             null|
|  10/13/2014|             null|             null|
|   5/22/2015|         8/9/2016|         8/9/2016|
|  10/13/2014|             null|             null|
|  10/13/2014|        2/24/2015|             null|
|  10/13/2014|             null|             null|
|  10/13/2014|             null|             null|
|  10/13/2014|             null|             null|
|  10/13/2014|             null

In [5]:
#Formateo de string a date, utilizando formateo a 'm/d/yyyy' debido a que encontramos meses sin cero al inicio
#Esto se almacena en un nuevo dataframe llamado df_stations_fixed
df_stations_fixed = df_stations_raw.withColumn ("install_date", to_date (col("install_date"),'M/d/yyyy'))\
                          .withColumn ("modification_date", to_date (col("modification_date"),'M/d/yyyy'))\
                          .withColumn ("decommission_date", to_date (col("decommission_date"),'M/d/yyyy'))

# FINALIZACION DE TRABAJO CON GENERANDO UN DATAFRAME LLAMADO df_stations_fixed

In [6]:
#Se hace un print schema y se eligio nombre df_stations_fixed debido a que no hay mas limpieza a hacer en este dataframe
df_stations_fixed.printSchema()


root
 |-- station_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- install_date: date (nullable = true)
 |-- install_dockcount: integer (nullable = true)
 |-- modification_date: date (nullable = true)
 |-- current_dockcount: integer (nullable = true)
 |-- decommission_date: date (nullable = true)



# INICIO TRABAJO CON WEATHER CSV

In [7]:
#Importacion weather datos
df_weather_raw = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('InferSchema','true')\
  .option('header', 'true')\
  .load(cos.url('weather.csv', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))
df_weather_raw.take(2)

[Row(Date='10/13/2014', Max_Temperature_F=71, Mean_Temperature_F=62, Min_TemperatureF=54, Max_Dew_Point_F=55, MeanDew_Point_F=51, Min_Dewpoint_F=46, Max_Humidity=87, Mean_Humidity=68, Min_Humidity=46, Max_Sea_Level_Pressure_In=30.03, Mean_Sea_Level_Pressure_In=29.79, Min_Sea_Level_Pressure_In=29.65, Max_Visibility_Miles=10, Mean_Visibility_Miles=10, Min_Visibility_Miles=4, Max_Wind_Speed_MPH=13, Mean_Wind_Speed_MPH=4, Max_Gust_Speed_MPH='21', Precipitation_In=0.0, Events='Rain'),
 Row(Date='10/14/2014', Max_Temperature_F=63, Mean_Temperature_F=59, Min_TemperatureF=55, Max_Dew_Point_F=52, MeanDew_Point_F=51, Min_Dewpoint_F=50, Max_Humidity=88, Mean_Humidity=78, Min_Humidity=63, Max_Sea_Level_Pressure_In=29.84, Mean_Sea_Level_Pressure_In=29.75, Min_Sea_Level_Pressure_In=29.54, Max_Visibility_Miles=10, Mean_Visibility_Miles=9, Min_Visibility_Miles=3, Max_Wind_Speed_MPH=10, Mean_Wind_Speed_MPH=5, Max_Gust_Speed_MPH='17', Precipitation_In=0.11, Events='Rain')]

In [8]:
#Revision de data schema
df_weather_raw.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Max_Temperature_F: integer (nullable = true)
 |-- Mean_Temperature_F: integer (nullable = true)
 |-- Min_TemperatureF: integer (nullable = true)
 |-- Max_Dew_Point_F: integer (nullable = true)
 |-- MeanDew_Point_F: integer (nullable = true)
 |-- Min_Dewpoint_F: integer (nullable = true)
 |-- Max_Humidity: integer (nullable = true)
 |-- Mean_Humidity: integer (nullable = true)
 |-- Min_Humidity: integer (nullable = true)
 |-- Max_Sea_Level_Pressure_In: double (nullable = true)
 |-- Mean_Sea_Level_Pressure_In: double (nullable = true)
 |-- Min_Sea_Level_Pressure_In: double (nullable = true)
 |-- Max_Visibility_Miles: integer (nullable = true)
 |-- Mean_Visibility_Miles: integer (nullable = true)
 |-- Min_Visibility_Miles: integer (nullable = true)
 |-- Max_Wind_Speed_MPH: integer (nullable = true)
 |-- Mean_Wind_Speed_MPH: integer (nullable = true)
 |-- Max_Gust_Speed_MPH: string (nullable = true)
 |-- Precipitation_In: double (nullable = tru

In [9]:
df_weather_raw.describe().show()

+-------+--------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+-------------------------+--------------------------+-------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+-------------------+------+
|summary|    Date| Max_Temperature_F|Mean_Temperature_F| Min_TemperatureF|  Max_Dew_Point_F|  MeanDew_Point_F|    Min_Dewpoint_F|     Max_Humidity|     Mean_Humidity|      Min_Humidity|Max_Sea_Level_Pressure_In|Mean_Sea_Level_Pressure_In|Min_Sea_Level_Pressure_In|Max_Visibility_Miles|Mean_Visibility_Miles|Min_Visibility_Miles|Max_Wind_Speed_MPH|Mean_Wind_Speed_MPH|Max_Gust_Speed_MPH|   Precipitation_In|Events|
+-------+--------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------

In [10]:
#Formateo de fechas
df_weather_fechas = df_weather_raw.withColumn ("Date", to_date (col("Date"),'M/d/yyyy'))

In [11]:
#Comprobacion de formateo de fechas exitoso
df_weather_fechas.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Max_Temperature_F: integer (nullable = true)
 |-- Mean_Temperature_F: integer (nullable = true)
 |-- Min_TemperatureF: integer (nullable = true)
 |-- Max_Dew_Point_F: integer (nullable = true)
 |-- MeanDew_Point_F: integer (nullable = true)
 |-- Min_Dewpoint_F: integer (nullable = true)
 |-- Max_Humidity: integer (nullable = true)
 |-- Mean_Humidity: integer (nullable = true)
 |-- Min_Humidity: integer (nullable = true)
 |-- Max_Sea_Level_Pressure_In: double (nullable = true)
 |-- Mean_Sea_Level_Pressure_In: double (nullable = true)
 |-- Min_Sea_Level_Pressure_In: double (nullable = true)
 |-- Max_Visibility_Miles: integer (nullable = true)
 |-- Mean_Visibility_Miles: integer (nullable = true)
 |-- Min_Visibility_Miles: integer (nullable = true)
 |-- Max_Wind_Speed_MPH: integer (nullable = true)
 |-- Mean_Wind_Speed_MPH: integer (nullable = true)
 |-- Max_Gust_Speed_MPH: string (nullable = true)
 |-- Precipitation_In: double (nullable = true)

In [12]:
#Se muestra los valores distintos de la columna Events
df_weather_fechas.select("Events").distinct().show()

+-------------------+
|             Events|
+-------------------+
|          Rain-Snow|
|               null|
|         Fog , Rain|
|                Fog|
|        Rain , Snow|
|           Fog-Rain|
|Rain , Thunderstorm|
|               Rain|
|               Snow|
|  Rain-Thunderstorm|
+-------------------+



In [13]:
#Reemplazamos los nulos en columna Events por Normal. Ya se consulto que para eventos que fueran nulos y/o vacíos se sobreentendía que era un día sin inclemencia meteorologica
df_weather_NoNAevent = df_weather_fechas.na.fill("Normal", ["Events"])
df_weather_NoNAevent.select("Events").distinct().show()

+-------------------+
|             Events|
+-------------------+
|          Rain-Snow|
|         Fog , Rain|
|                Fog|
|        Rain , Snow|
|           Fog-Rain|
|Rain , Thunderstorm|
|               Rain|
|               Snow|
|  Rain-Thunderstorm|
|             Normal|
+-------------------+



In [14]:
#df_weather_fix = df_weather_NoNAevent.withColumn(COLUMNA, regexp_replace(COLUMNA, VALOR A REEMPLAZAR, NUEVO VALOR))
#Se estandariza los valores que tenian espacios o fueron ingresados de otra manera

df_weather_fixed = df_weather_NoNAevent.withColumn('Events', regexp_replace('Events', 'Fog , Rain', 'Fog-Rain'))\
                                     .withColumn('Events', regexp_replace('Events', 'Rain , Snow', 'Rain-Snow'))\
                                     .withColumn('Events', regexp_replace('Events', 'Rain , Thunderstorm', 'Rain-Thunderstorm'))

#Se comprueba la columna Events haya sido formateada exitosamente
df_weather_fixed.select("Events").distinct().show()

+-----------------+
|           Events|
+-----------------+
|        Rain-Snow|
|              Fog|
|         Fog-Rain|
|             Rain|
|             Snow|
|Rain-Thunderstorm|
|           Normal|
+-----------------+



In [15]:
#Contamos con 1 registro nulo en Mean_Temperature_F el cual va a ser reemplazado por un valor numerico para evitar inconvenientes
#Contamos 504 valores únicamente en la columna Max_Gust_Speed_MPH, a revisar
df_weather_fixed.describe().show()

+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+-------------------------+--------------------------+-------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+-------------------+------+
|summary| Max_Temperature_F|Mean_Temperature_F| Min_TemperatureF|  Max_Dew_Point_F|  MeanDew_Point_F|    Min_Dewpoint_F|     Max_Humidity|     Mean_Humidity|      Min_Humidity|Max_Sea_Level_Pressure_In|Mean_Sea_Level_Pressure_In|Min_Sea_Level_Pressure_In|Max_Visibility_Miles|Mean_Visibility_Miles|Min_Visibility_Miles|Max_Wind_Speed_MPH|Mean_Wind_Speed_MPH|Max_Gust_Speed_MPH|   Precipitation_In|Events|
+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+--------------

In [16]:
#Genero un dataset temporal bajo el nombre DF1 creando 

#Genero una nueva columna calculando la media en base a las columnas TempMax y TempMin
df1=df_weather_fixed.withColumn("mean_of_col", (col("Max_Temperature_F")+col("Min_TemperatureF"))/2)
df1.select('Date','Max_Temperature_F','Mean_Temperature_F','Min_TemperatureF','mean_of_col').show(5)
#Max_Temperature_F
#Mean_Temperature_F
#Min_TemperatureF


+----------+-----------------+------------------+----------------+-----------+
|      Date|Max_Temperature_F|Mean_Temperature_F|Min_TemperatureF|mean_of_col|
+----------+-----------------+------------------+----------------+-----------+
|2014-10-13|               71|                62|              54|       62.5|
|2014-10-14|               63|                59|              55|       59.0|
|2014-10-15|               62|                58|              54|       58.0|
|2014-10-16|               71|                61|              52|       61.5|
|2014-10-17|               64|                60|              57|       60.5|
+----------+-----------------+------------------+----------------+-----------+
only showing top 5 rows



In [17]:
#Se procede a reemplazar los nulos en la columna Mean_Temperature_F por valor en la columna mean_of_col
df2=df1.withColumn('Mean_Temperature_F', coalesce('Mean_Temperature_F', 'mean_of_col'))
df2.describe().show()



+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+-------------------------+--------------------------+-------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+-------------------+------+-----------------+
|summary| Max_Temperature_F|Mean_Temperature_F| Min_TemperatureF|  Max_Dew_Point_F|  MeanDew_Point_F|    Min_Dewpoint_F|     Max_Humidity|     Mean_Humidity|      Min_Humidity|Max_Sea_Level_Pressure_In|Mean_Sea_Level_Pressure_In|Min_Sea_Level_Pressure_In|Max_Visibility_Miles|Mean_Visibility_Miles|Min_Visibility_Miles|Max_Wind_Speed_MPH|Mean_Wind_Speed_MPH|Max_Gust_Speed_MPH|   Precipitation_In|Events|      mean_of_col|
+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+----------------

In [18]:
#Se genera un nuevo dataframe llamado df_weather_fixed2 haciendo un drop de 'mean of col'
df_weather_fixed2=df2.drop(col("mean_of_col"))
df_weather_fixed2.describe().show()

+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+-------------------------+--------------------------+-------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+-------------------+------+
|summary| Max_Temperature_F|Mean_Temperature_F| Min_TemperatureF|  Max_Dew_Point_F|  MeanDew_Point_F|    Min_Dewpoint_F|     Max_Humidity|     Mean_Humidity|      Min_Humidity|Max_Sea_Level_Pressure_In|Mean_Sea_Level_Pressure_In|Min_Sea_Level_Pressure_In|Max_Visibility_Miles|Mean_Visibility_Miles|Min_Visibility_Miles|Max_Wind_Speed_MPH|Mean_Wind_Speed_MPH|Max_Gust_Speed_MPH|   Precipitation_In|Events|
+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+--------------

In [19]:
#En cuanto a 'Max_Gust_Speed_MPH' se procedera a reemplazar los valores nulos o con guiones a convertirlos a cero.
#Basados en "https://www.inumet.gub.uy/sites/default/files/2018-04/Inumet_Informe_postevento_12abril2018.pdf"
#Podemos afirmar que la rafaga de viento es "el aumento breve y repentino de la velocidad del viento respecto a su valor medio, fuente OMM Nº182)"
#Fuente https://www.inumet.gub.uy/institucional/glosario
#Es un aumento repentino y significativo en las fluctuaciones de la velocidad del viento. La velocidad punta del viento debe alcanzar por lo menos 16 nudos (30 km/h / 18.64 mp/h) y la variación entre los picos y la calma es de por lo menos 10 nudos (18 km/h / 11.18 mp/h). Generalmente la duración es menor de 20 segundos.
#Esto refuerza el concepto que no siempre se presentaran rafagas de viento
#De manera que si el valor es cero en esta medida sabremos que no hubieron rafagas de viento. Si bien es cierto que podríamos cambiar esta columna a una clasificación de RAFAGA o SIN RAFAGA por ejemplo, mantendremos la data para estudios posteriores de ser necesario
df_weather_fixed2.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Max_Temperature_F: integer (nullable = true)
 |-- Mean_Temperature_F: double (nullable = true)
 |-- Min_TemperatureF: integer (nullable = true)
 |-- Max_Dew_Point_F: integer (nullable = true)
 |-- MeanDew_Point_F: integer (nullable = true)
 |-- Min_Dewpoint_F: integer (nullable = true)
 |-- Max_Humidity: integer (nullable = true)
 |-- Mean_Humidity: integer (nullable = true)
 |-- Min_Humidity: integer (nullable = true)
 |-- Max_Sea_Level_Pressure_In: double (nullable = true)
 |-- Mean_Sea_Level_Pressure_In: double (nullable = true)
 |-- Min_Sea_Level_Pressure_In: double (nullable = true)
 |-- Max_Visibility_Miles: integer (nullable = true)
 |-- Mean_Visibility_Miles: integer (nullable = true)
 |-- Min_Visibility_Miles: integer (nullable = true)
 |-- Max_Wind_Speed_MPH: integer (nullable = true)
 |-- Mean_Wind_Speed_MPH: integer (nullable = true)
 |-- Max_Gust_Speed_MPH: string (nullable = true)
 |-- Precipitation_In: double (nullable = true)


In [20]:
#Basados en el tipo de dato Max y Mean Speed conjunto a los valores existentes de tipo numerico, deberiamos parsear la columna a tipo Integer.
df_weather_fixed2.select("Max_Gust_Speed_MPH").distinct().show()

+------------------+
|Max_Gust_Speed_MPH|
+------------------+
|                29|
|                30|
|                34|
|                22|
|                28|
|                16|
|                52|
|              null|
|                31|
|                18|
|                17|
|                26|
|                46|
|                41|
|                23|
|                 -|
|                38|
|                25|
|                33|
|                24|
+------------------+
only showing top 20 rows



In [21]:
#Se cambian los valores con "-" a cero de la columna Max_Gust_Speed_MPH
df_weather_fixed2 = df_weather_fixed2.withColumn('Max_Gust_Speed_MPH', regexp_replace('Max_Gust_Speed_MPH', '-', '0'))
df_weather_fixed2.select("Max_Gust_Speed_MPH").distinct().show()

+------------------+
|Max_Gust_Speed_MPH|
+------------------+
|                29|
|                30|
|                34|
|                22|
|                28|
|                16|
|                52|
|                 0|
|              null|
|                31|
|                18|
|                17|
|                26|
|                46|
|                41|
|                23|
|                38|
|                25|
|                33|
|                24|
+------------------+
only showing top 20 rows



In [22]:
#Se cambian los valores nulos a cero de la columna Max_Gust_Speed_MPH
df_weather_fixed2 = df_weather_fixed2.na.fill("0", ["Max_Gust_Speed_MPH"])
df_weather_fixed2.select("Max_Gust_Speed_MPH").distinct().show()

+------------------+
|Max_Gust_Speed_MPH|
+------------------+
|                29|
|                30|
|                34|
|                22|
|                28|
|                16|
|                52|
|                 0|
|                31|
|                18|
|                17|
|                26|
|                46|
|                41|
|                23|
|                38|
|                25|
|                33|
|                24|
|                32|
+------------------+
only showing top 20 rows



In [23]:
#Se importa debido a que se tenia un error "NameError: name 'IntegerType' is not defined"
from pyspark.sql.types import IntegerType

#Se castea la columna de String a Integer
df_weather_fixed2=df_weather_fixed2.withColumn("Max_Gust_Speed_MPH",df_weather_fixed2.Max_Gust_Speed_MPH.cast(IntegerType()))
df_weather_fixed2.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Max_Temperature_F: integer (nullable = true)
 |-- Mean_Temperature_F: double (nullable = true)
 |-- Min_TemperatureF: integer (nullable = true)
 |-- Max_Dew_Point_F: integer (nullable = true)
 |-- MeanDew_Point_F: integer (nullable = true)
 |-- Min_Dewpoint_F: integer (nullable = true)
 |-- Max_Humidity: integer (nullable = true)
 |-- Mean_Humidity: integer (nullable = true)
 |-- Min_Humidity: integer (nullable = true)
 |-- Max_Sea_Level_Pressure_In: double (nullable = true)
 |-- Mean_Sea_Level_Pressure_In: double (nullable = true)
 |-- Min_Sea_Level_Pressure_In: double (nullable = true)
 |-- Max_Visibility_Miles: integer (nullable = true)
 |-- Mean_Visibility_Miles: integer (nullable = true)
 |-- Min_Visibility_Miles: integer (nullable = true)
 |-- Max_Wind_Speed_MPH: integer (nullable = true)
 |-- Mean_Wind_Speed_MPH: integer (nullable = true)
 |-- Max_Gust_Speed_MPH: integer (nullable = true)
 |-- Precipitation_In: double (nullable = true)

# SE FINALIZA LA LIMPIEZA DE WEATHER GENERANDO DATAFRAME df_weather_fixed2

In [24]:
#Todos los valores son validos y de tipo numerico en Max_Gust_Speed_MPH
df_weather_fixed2.select("Max_Gust_Speed_MPH").distinct().show(df_weather_fixed2.count())

+------------------+
|Max_Gust_Speed_MPH|
+------------------+
|                31|
|                34|
|                28|
|                26|
|                22|
|                52|
|                16|
|                20|
|                41|
|                37|
|                17|
|                23|
|                39|
|                38|
|                25|
|                24|
|                29|
|                21|
|                32|
|                33|
|                30|
|                46|
|                 0|
|                18|
|                36|
+------------------+



In [25]:
#Estaciones en USA
#fuente https://www.visittheusa.mx/info/tiempo-y-estaciones-del-ano
#primeravera 03-05
#verano 06-08
#otoño 09-11
#invierno 12 - 01

df_weather_fixed = df_weather_fixed2.withColumn ("temporada", when ( (month("Date")<=5) & (month("Date")>=3), "primavera").\
                                                otherwise(
                                                     when ( (month("Date")<=8) & (month("Date")>=6), "verano").\
                                                     otherwise(
                                                         when ( (month("Date")<=11) & (month("Date")>=9), "otoño").\
                                                         otherwise(
                                                             "invierno"
                                                         )
                                                     )
                                                )
)


df_weather_fixed.select("temporada").distinct().show()

+---------+
|temporada|
+---------+
|   verano|
|primavera|
|    otoño|
| invierno|
+---------+



In [26]:
df_weather_fixed.describe().show()

+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+-------------------------+--------------------------+-------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+-------------------+------+---------+
|summary| Max_Temperature_F|Mean_Temperature_F| Min_TemperatureF|  Max_Dew_Point_F|  MeanDew_Point_F|    Min_Dewpoint_F|     Max_Humidity|     Mean_Humidity|      Min_Humidity|Max_Sea_Level_Pressure_In|Mean_Sea_Level_Pressure_In|Min_Sea_Level_Pressure_In|Max_Visibility_Miles|Mean_Visibility_Miles|Min_Visibility_Miles|Max_Wind_Speed_MPH|Mean_Wind_Speed_MPH|Max_Gust_Speed_MPH|   Precipitation_In|Events|temporada|
+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+-------------

# Fin de trabajo de correcion de weather.csv
#df_stations_fixed
#df_weather_fixed


# Inicio de trabajo con trip.csv

In [27]:

df_trip_raw = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('InferSchema','true')\
    .option('header', 'true')\
  .load(cos.url('trip.csv', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))
df_trip_raw.take(5)


[Row(trip_id=431, starttime='10/13/2014 10:31', stoptime='10/13/2014 10:48', bikeid='SEA00298', tripduration=985.935, from_station_name='2nd Ave & Spring St', to_station_name='Occidental Park / Occidental Ave S & S Washington St', from_station_id='CBD-06', to_station_id='PS-04', usertype='Member', gender='Male', birthyear='1960'),
 Row(trip_id=432, starttime='10/13/2014 10:32', stoptime='10/13/2014 10:48', bikeid='SEA00195', tripduration=926.375, from_station_name='2nd Ave & Spring St', to_station_name='Occidental Park / Occidental Ave S & S Washington St', from_station_id='CBD-06', to_station_id='PS-04', usertype='Member', gender='Male', birthyear='1970'),
 Row(trip_id=433, starttime='10/13/2014 10:33', stoptime='10/13/2014 10:48', bikeid='SEA00486', tripduration=883.831, from_station_name='2nd Ave & Spring St', to_station_name='Occidental Park / Occidental Ave S & S Washington St', from_station_id='CBD-06', to_station_id='PS-04', usertype='Member', gender='Female', birthyear='1988'),

In [28]:
#Se contabiliza la cantidad de registros en la tabla de trips
df_trip_raw.count()

286858

In [29]:
#Se contabiliza la cantidad de registros unicos en la tabla de trips
df_trip_raw.distinct().count()

236066

In [30]:
df_trip_raw.printSchema()

root
 |-- trip_id: integer (nullable = true)
 |-- starttime: string (nullable = true)
 |-- stoptime: string (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- tripduration: double (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- from_station_id: string (nullable = true)
 |-- to_station_id: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: string (nullable = true)



In [31]:
#Hay un trip id que se repite con registros distintos
df_trip_raw.select("trip_id").distinct().count()

236065

In [32]:
#Se utiliza la funcion cache para aumentar eficiencia del procesamiento
df_trips_deduc = df_trip_raw.distinct()
df_trips_deduc.cache()

DataFrame[trip_id: int, starttime: string, stoptime: string, bikeid: string, tripduration: double, from_station_name: string, to_station_name: string, from_station_id: string, to_station_id: string, usertype: string, gender: string, birthyear: string]

In [33]:
#Se listan los trip_ids conjunto a su count
#De manera se identifica que trip_id está repetido
df_trips_deduc.groupBy ("trip_id").agg(count("*").alias("cantidad")).orderBy(desc("cantidad")).show()

+-------+--------+
|trip_id|cantidad|
+-------+--------+
|  59000|       2|
| 220377|       1|
| 166644|       1|
| 213962|       1|
|  43256|       1|
|  82385|       1|
| 145747|       1|
| 164704|       1|
| 183920|       1|
| 193728|       1|
| 216144|       1|
| 225626|       1|
| 248280|       1|
|  40687|       1|
|  50732|       1|
|  51622|       1|
| 101740|       1|
| 126385|       1|
| 157712|       1|
| 168235|       1|
+-------+--------+
only showing top 20 rows



In [34]:
#Se muestran los registros con ID 59000, encontramos que el registro con gender stoptime es incorrect, deberia borrarse
df_trips_deduc.filter("trip_id=59000").show()

+-------+---------------+---------------+--------+------------+--------------------+--------------------+---------------+----------------+--------------------+--------+---------+
|trip_id|      starttime|       stoptime|  bikeid|tripduration|   from_station_name|     to_station_name|from_station_id|   to_station_id|            usertype|  gender|birthyear|
+-------+---------------+---------------+--------+------------+--------------------+--------------------+---------------+----------------+--------------------+--------+---------+
|  59000|4/17/2015 14:21|4/17/2015 19:21|SEA00362|   17990.668|6th Ave S & S Kin...|Westlake Ave & 6t...|          ID-04|"SLU-15"trip_id"|           starttime|stoptime|   bikeid|
|  59000|4/17/2015 14:21|4/17/2015 19:21|SEA00362|   17990.668|6th Ave S & S Kin...|Westlake Ave & 6t...|          ID-04|          SLU-15|Short-Term Pass H...|    null|     null|
+-------+---------------+---------------+--------+------------+--------------------+--------------------+

In [35]:
df_eliminar = df_trips_deduc.filter ("trip_id = '59000' and gender = 'stoptime'")
df_eliminar.show()

+-------+---------------+---------------+--------+------------+--------------------+--------------------+---------------+----------------+---------+--------+---------+
|trip_id|      starttime|       stoptime|  bikeid|tripduration|   from_station_name|     to_station_name|from_station_id|   to_station_id| usertype|  gender|birthyear|
+-------+---------------+---------------+--------+------------+--------------------+--------------------+---------------+----------------+---------+--------+---------+
|  59000|4/17/2015 14:21|4/17/2015 19:21|SEA00362|   17990.668|6th Ave S & S Kin...|Westlake Ave & 6t...|          ID-04|"SLU-15"trip_id"|starttime|stoptime|   bikeid|
+-------+---------------+---------------+--------+------------+--------------------+--------------------+---------------+----------------+---------+--------+---------+



In [36]:
#Iguala a df_trip_raw.select("trip_id").distinct().count()
#Lo cual verifica que fue eliminado el registro con trip_id duplicado
df_trips_uniqueids = df_trips_deduc.subtract(df_eliminar)
df_trips_uniqueids.count()

236065

In [37]:
#StarTime, StopTime deberian ser DateTime
#birthyear deberia ser Integer (ya que no deben haber años con decimales)
#Resta aun sanear los datos que tengan como gender stoptime
df_trips_uniqueids.printSchema()

root
 |-- trip_id: integer (nullable = true)
 |-- starttime: string (nullable = true)
 |-- stoptime: string (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- tripduration: double (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- from_station_id: string (nullable = true)
 |-- to_station_id: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: string (nullable = true)



In [38]:
#Verificacion de tipo de dato de startime y stoptime
df_trips_uniqueids.select("starttime","stoptime").show(5)

+----------------+----------------+
|       starttime|        stoptime|
+----------------+----------------+
|12/31/2014 11:38|12/31/2014 11:54|
| 7/26/2015 14:24| 7/26/2015 14:54|
| 8/18/2015 10:04| 8/18/2015 10:05|
| 9/15/2015 13:29| 9/15/2015 13:43|
| 6/20/2016 20:42| 6/20/2016 20:45|
+----------------+----------------+
only showing top 5 rows



In [39]:
#Se elige "M/d/yyyy H:m" para asegurar que no haya errores
#En este caso ya conocemos que el starttime y el stoptime datan del mismo dia
#Se genera un nuevo dataframe llamado df_trips_fixed
df_trips_fixed = df_trips_uniqueids.withColumn("starttime",to_timestamp(col("starttime"),'M/d/yyyy H:m')).\
                                    withColumn("stoptime",to_timestamp(col("stoptime"),'M/d/yyyy H:m')).\
                                    withColumn("Date",to_date(col("starttime")))

#Se comprueba que startime y stoptime sean datetime y date sea date
df_trips_fixed.printSchema()        

root
 |-- trip_id: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- tripduration: double (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- from_station_id: string (nullable = true)
 |-- to_station_id: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: string (nullable = true)
 |-- Date: date (nullable = true)



In [40]:
#Luego de transformar los datos correctamente y agregar columna date
#Reviso que si hay presencia de trips con mayor startime a stoptime
#De esta manera podemos asegurarnos de no necesitar crear una segunda columna auxiliar de date, para casos de viajes que empiece por ejemplo dia 1 y terminen dia 2, experimentando mas de un clima
#En caso de hallar viajes con startime mayor al stoptime deberan ser eliminados (viajes que difieren a nivel hora, ejemplo startime es 1 de abril a las 14 horas y stoptime es 1 de abril a las 13 horas)
df_temp = df_trips_fixed.filter(col("starttime") > col("stoptime"))
df_temp.select("trip_id","starttime","stoptime").show()

+-------+-------------------+-------------------+
|trip_id|          starttime|           stoptime|
+-------+-------------------+-------------------+
|   8669|2014-11-02 01:44:00|2014-11-02 01:04:00|
|   8660|2014-11-02 01:29:00|2014-11-02 01:12:00|
|   8666|2014-11-02 01:31:00|2014-11-02 01:11:00|
|   8663|2014-11-02 01:29:00|2014-11-02 01:11:00|
|   8670|2014-11-02 01:52:00|2014-11-02 01:07:00|
|   8667|2014-11-02 01:37:00|2014-11-02 01:12:00|
|   8672|2014-11-02 01:59:00|2014-11-02 01:25:00|
+-------+-------------------+-------------------+



In [41]:
#Cantidad registros sin eliminar el subgrupo de starttime > stoptime
df_trips_fixed.count()

236065

In [42]:
#Cantidad de registros a eliminar
df_temp.count()

7

In [43]:
#236065 - 7 = 236058 registros deberia resultar de eliminar registros con startime mayor a stoptime
df_trips_fixed = df_trips_fixed.subtract(df_temp)
df_trips_fixed.count()

236058

In [44]:
#Revision schema df_stations fixed
df_trips_fixed.printSchema()

root
 |-- trip_id: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- tripduration: double (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- from_station_id: string (nullable = true)
 |-- to_station_id: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: string (nullable = true)
 |-- Date: date (nullable = true)



In [45]:
#revision gender
df_trips_fixed.select("gender").distinct().show()
#En base a consulta realizada se toma como valido el null, indica que el usuario no estuvo dispuesto a responder este dato

+------+
|gender|
+------+
|  null|
|Female|
| Other|
|  Male|
+------+



In [46]:
#revision usertype
df_trips_fixed.select("usertype").distinct().show()
#contamos con los datos correctos

+--------------------+
|            usertype|
+--------------------+
|Short-Term Pass H...|
|              Member|
+--------------------+



In [47]:
#revision birthyear para un posterior parseo
df_trips_fixed.select("birthyear").distinct().show()
#Notamos que consta de valores numericos y nulos, notando que algunos usuarios no estaban dispuestos a brindar la informacion

+---------+
|birthyear|
+---------+
|     1953|
|     1957|
|     1987|
|     1956|
|     1936|
|     1958|
|     1943|
|     1972|
|     1931|
|     1988|
|     1977|
|     1971|
|     1984|
|     1982|
|     1965|
|     1962|
|     1954|
|     null|
|     1981|
|     1978|
+---------+
only showing top 20 rows



In [48]:
#df_weather_fixed2=df_weather_fixed2.withColumn("Max_Gust_Speed_MPH",df_weather_fixed2.Max_Gust_Speed_MPH.cast(IntegerType()))
#df_weather_fixed2.printSchema()

df_trips_fixed=df_trips_fixed.withColumn("birthyear",df_trips_fixed.birthyear.cast(IntegerType()))
df_trips_fixed.printSchema()

root
 |-- trip_id: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- tripduration: double (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- from_station_id: string (nullable = true)
 |-- to_station_id: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: integer (nullable = true)
 |-- Date: date (nullable = true)



In [49]:
#Revision de valores de birthyear posterior al parseo
df_trips_fixed.select("birthyear").distinct().show()

+---------+
|birthyear|
+---------+
|     1959|
|     1990|
|     1975|
|     1977|
|     1974|
|     1955|
|     1978|
|     1961|
|     1942|
|     1939|
|     1944|
|     null|
|     1952|
|     1956|
|     1988|
|     1997|
|     1994|
|     1968|
|     1951|
|     1973|
+---------+
only showing top 20 rows



In [50]:
#Sigo contando con 236058 registros
df_trips_fixed.count()

236058

In [51]:
#Creando edad del user al momento de hacer el viaje

#todays_year = date.today().year
df_trips_fixed = df_trips_fixed.withColumn ("age",year("starttime")-col("birthyear"))

#df_trips_fixed = df_trips_fixed.withColumn("age", ((year(col("Date"))) - (year(col("birthyear")))))

df_trips_fixed.printSchema()

root
 |-- trip_id: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- tripduration: double (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- from_station_id: string (nullable = true)
 |-- to_station_id: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- age: integer (nullable = true)



In [52]:
#Se muestra todas las edades calculadas
df_trips_fixed.select("age").distinct().show(df_trips_fixed.select("age").distinct().count())

+----+
| age|
+----+
|  31|
|  85|
|  65|
|  53|
|  78|
|  34|
|  28|
|  76|
|  26|
|  27|
|  44|
|  22|
|  47|
|null|
|  52|
|  16|
|  40|
|  20|
|  57|
|  54|
|  48|
|  19|
|  64|
|  41|
|  43|
|  37|
|  61|
|  17|
|  72|
|  35|
|  55|
|  59|
|  23|
|  39|
|  49|
|  51|
|  69|
|  63|
|  77|
|  50|
|  45|
|  38|
|  80|
|  25|
|  73|
|  24|
|  70|
|  62|
|  29|
|  21|
|  32|
|  60|
|  56|
|  58|
|  33|
|  68|
|  71|
|  42|
|  79|
|  30|
|  66|
|  46|
|  67|
|  18|
|  36|
+----+



In [53]:
#Se agrega una columna de quarter en base a la install date la cual no tiene nulos. Esto ayudara a un posterior analisis de progresion de creacion de estaciones en caso de realizarse
df_stations_fixed=df_stations_fixed.withColumn("quarter_install_date",quarter("install_date"))
df_stations_fixed.printSchema()
df_stations_fixed.describe().show()
#LAT y LONG estan en double, reminder propio

root
 |-- station_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- install_date: date (nullable = true)
 |-- install_dockcount: integer (nullable = true)
 |-- modification_date: date (nullable = true)
 |-- current_dockcount: integer (nullable = true)
 |-- decommission_date: date (nullable = true)
 |-- quarter_install_date: integer (nullable = true)

+-------+----------+--------------------+-----------------+--------------------+------------------+------------------+--------------------+
|summary|station_id|                name|              lat|                long| install_dockcount| current_dockcount|quarter_install_date|
+-------+----------+--------------------+-----------------+--------------------+------------------+------------------+--------------------+
|  count|        58|                  58|               58|                  58|                58|                58|                  

In [54]:
#Se genera el quarter en base a la columna generada de fecha, "Date"
df_trips_fixed=df_trips_fixed.withColumn("quarter_date",quarter("Date"))

df_trips_fixed.printSchema()
df_trips_fixed.describe().show()

root
 |-- trip_id: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- tripduration: double (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- from_station_id: string (nullable = true)
 |-- to_station_id: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- quarter_date: integer (nullable = true)

+-------+------------------+--------+------------------+--------------------+--------------------+---------------+-------------+--------------------+------+------------------+------------------+------------------+
|summary|           trip_id|  bikeid|      tripduration|   from_station_name|     to_station_name|from_station_id|to_station_id|            usertype

In [55]:
df_trips_fixed= df_trips_fixed.withColumn("age_range", when((col("age") <= 25) & (col("age") >= 1), "jovenes").\
                                  otherwise(when((col("age") <= 45) & (col("age") >= 26), "adulto").\
                                  otherwise(when((col("age") <= 100) & (col("age") >= 46), "adulto_mayor").\
                                  otherwise("sin_edad"))))

df_trips_fixed.select("age_range").distinct().show()

+------------+
|   age_range|
+------------+
|    sin_edad|
|     jovenes|
|      adulto|
|adulto_mayor|
+------------+



In [56]:
#En caso de querer recalcular los segundos y luego en base a ello los minutos. Aun asi habiendo revisado no hay gran diferencia entre los segundos calculados y los segundos de tripduration

#Generacion de columna de duracion de viaje en segundos y luego en minutos
#from pyspark.sql.types import LongType
df_trips_fixed=df_trips_fixed.withColumn('trip_duration_seconds',unix_timestamp("stoptime") - unix_timestamp('starttime')) \
                             .withColumn('trip_duration_minutes',round(col('trip_duration_seconds')/60))

#Se eliminara tripduration al ser de tipo interval y no poder guardarlo con parque pero se crea un nuevo calculo
df_trips_fixed=df_trips_fixed.drop(col("tripduration"))

df_trips_fixed.select('stoptime','starttime','trip_duration_seconds','trip_duration_minutes').show(5)
df_trips_fixed.printSchema()

+-------------------+-------------------+---------------------+---------------------+
|           stoptime|          starttime|trip_duration_seconds|trip_duration_minutes|
+-------------------+-------------------+---------------------+---------------------+
|2016-04-18 22:15:00|2016-04-18 21:36:00|                 2340|                 39.0|
|2015-06-07 21:45:00|2015-06-07 21:22:00|                 1380|                 23.0|
|2016-07-18 17:14:00|2016-07-18 17:06:00|                  480|                  8.0|
|2015-07-20 16:10:00|2015-07-20 14:13:00|                 7020|                117.0|
|2015-06-10 16:33:00|2015-06-10 16:02:00|                 1860|                 31.0|
+-------------------+-------------------+---------------------+---------------------+
only showing top 5 rows

root
 |-- trip_id: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- from_station_name: string 

# FIN DE TRABAJO EN DATEFRAME DE TRIPS

#DATAFRAMES FINALES
#df_stations_fixed
#df_weather_fixed
#df_trips_fixed

In [57]:
print ("Dataframe Stations Schema")
df_stations_fixed.printSchema()
print ("Dataframe Weather Schema")
df_weather_fixed.printSchema()
print ("Dataframe Trips Schema")
df_trips_fixed.printSchema()

Dataframe Stations Schema
root
 |-- station_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- install_date: date (nullable = true)
 |-- install_dockcount: integer (nullable = true)
 |-- modification_date: date (nullable = true)
 |-- current_dockcount: integer (nullable = true)
 |-- decommission_date: date (nullable = true)
 |-- quarter_install_date: integer (nullable = true)

Dataframe Weather Schema
root
 |-- Date: date (nullable = true)
 |-- Max_Temperature_F: integer (nullable = true)
 |-- Mean_Temperature_F: double (nullable = true)
 |-- Min_TemperatureF: integer (nullable = true)
 |-- Max_Dew_Point_F: integer (nullable = true)
 |-- MeanDew_Point_F: integer (nullable = true)
 |-- Min_Dewpoint_F: integer (nullable = true)
 |-- Max_Humidity: integer (nullable = true)
 |-- Mean_Humidity: integer (nullable = true)
 |-- Min_Humidity: integer (nullable = true)
 |-- Max_Sea_Level_Pressure_In: double

In [58]:
#Joins, join por defecto usado, inner join
df_trips_weather = df_trips_fixed.join (df_weather_fixed,"Date")
df_trips_weather.describe().show()


+-------+------------------+--------+--------------------+--------------------+---------------+-------------+--------------------+------+------------------+------------------+------------------+---------+---------------------+---------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------------+--------------------------+-------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+-------------------+------+---------+
|summary|           trip_id|  bikeid|   from_station_name|     to_station_name|from_station_id|to_station_id|            usertype|gender|         birthyear|               age|      quarter_date|age_range|trip_duration_seconds|trip_duration_minutes| Max_Temperature_F|Mean_Temperature_F|  Min_TemperatureF|   Max_Dew_Point_F|   MeanDew_Point_F|

# Guardar df_trips_weather
#se utiliza el directorio que se desprende de las credenciales al ingresar los datos csv

In [59]:
df_trips_weather.write.mode("overwrite").parquet(cos.url('trips_weather', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))

In [60]:
df_trips_from_station = df_trips_fixed.join (df_stations_fixed, df_stations_fixed.station_id == df_trips_fixed.from_station_id, "inner")
df_trips_from_station.describe().show()

+-------+-----------------+--------+--------------------+--------------------+---------------+-------------+--------------------+------+------------------+------------------+------------------+---------+---------------------+---------------------+----------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+
|summary|          trip_id|  bikeid|   from_station_name|     to_station_name|from_station_id|to_station_id|            usertype|gender|         birthyear|               age|      quarter_date|age_range|trip_duration_seconds|trip_duration_minutes|station_id|                name|                 lat|                long| install_dockcount| current_dockcount|quarter_install_date|
+-------+-----------------+--------+--------------------+--------------------+---------------+-------------+--------------------+------+------------------+------------------+------------------+---------+---------------------+-------------

In [61]:
df_trips_from_station.show(5)

+-------+-------------------+-------------------+--------+--------------------+--------------------+---------------+-------------+--------------------+------+---------+----------+----+------------+------------+---------------------+---------------------+----------+--------------------+---------+-----------+------------+-----------------+-----------------+-----------------+-----------------+--------------------+
|trip_id|          starttime|           stoptime|  bikeid|   from_station_name|     to_station_name|from_station_id|to_station_id|            usertype|gender|birthyear|      Date| age|quarter_date|   age_range|trip_duration_seconds|trip_duration_minutes|station_id|                name|      lat|       long|install_date|install_dockcount|modification_date|current_dockcount|decommission_date|quarter_install_date|
+-------+-------------------+-------------------+--------+--------------------+--------------------+---------------+-------------+--------------------+------+---------+--

In [62]:
df_trips_to_station = df_trips_fixed.join (df_stations_fixed, df_stations_fixed.station_id == df_trips_fixed.to_station_id, "inner")

df_trips_to_station.describe().show()

+-------+------------------+--------+--------------------+--------------------+---------------+-------------+--------------------+------+------------------+------------------+------------------+---------+---------------------+---------------------+----------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+
|summary|           trip_id|  bikeid|   from_station_name|     to_station_name|from_station_id|to_station_id|            usertype|gender|         birthyear|               age|      quarter_date|age_range|trip_duration_seconds|trip_duration_minutes|station_id|                name|                 lat|                long| install_dockcount| current_dockcount|quarter_install_date|
+-------+------------------+--------+--------------------+--------------------+---------------+-------------+--------------------+------+------------------+------------------+------------------+---------+---------------------+----------

In [63]:
#Guardo los dos nuevos joins creados y los dataframe fix finales
#df_trips_to_station
#df_trips_from_station
#df_stations_fixed
#df_weather_fixed
#df_trips_fixed

df_trips_to_station.write.mode("overwrite").parquet(cos.url('trips_to_station', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))
df_trips_from_station.write.mode("overwrite").parquet(cos.url('trips_from_station', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))
df_stations_fixed.write.mode("overwrite").parquet(cos.url('stations_fixed', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))
df_weather_fixed.write.mode("overwrite").parquet(cos.url('weather_fixed', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))
df_trips_fixed.write.mode("overwrite").parquet(cos.url('trips_fixed', 'tallerdadatasciencecamiloferreira-donotdelete-pr-fz0ndvhao4eorr'))

In [64]:
print("Joins guardados\ntrips_weather\ntrips_to_station\ntrips_from_station")

print("Dataframes limpio y modificados\nstations_fixed\nweather_fixed\ntrips_fixed")

Joins guardados
trips_weather
trips_to_station
trips_from_station
Dataframes limpio y modificados
stations_fixed
weather_fixed
trips_fixed
